In [2]:
import os
from shutil import rmtree, copy
from glob import glob
from random import sample

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
DATA_MOD_DIR = DATA_DIR + '_mod'

def get_classes():
    return {
                i: os.path.split(name)[-1] for i, name in
                enumerate(sorted(glob(os.path.join(DATA_DIR, 'test', '*'))))
           }

def make_data_mod_uni(files_per_class, refresh):
    if os.path.exists(DATA_MOD_DIR) and not refresh:
        return

    if os.path.exists(DATA_MOD_DIR):
        rmtree(DATA_MOD_DIR)
    
    for name in get_classes().values():
        os.makedirs(os.path.join(DATA_MOD_DIR, name))
        files_to_copy = glob(os.path.join(DATA_DIR, 'train', name, '*'))
        files_to_copy = sample(files_to_copy, min(files_per_class, len(files_to_copy)))
        for f in files_to_copy:
            copy(f, os.path.join(DATA_MOD_DIR, name))
    return            

make_data_mod_uni(25, refresh=False)

In [3]:
import numpy as np
import tensorflow as tf

RAW_IMAGE_HEIGHT = 224
RAW_IMAGE_WIDTH = 224
RAW_IMAGE_CHANNELS = 3

batch_size = 128
dataset_mod_train = tf.keras.utils.image_dataset_from_directory(
    DATA_MOD_DIR,
    batch_size=batch_size,
    label_mode='categorical',
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True,
)
dataset_valid = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'valid'),
    batch_size=batch_size,
    label_mode='categorical',
    image_size=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH),
    crop_to_aspect_ratio=True
)

2023-10-08 21:51:07.571895: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 21:51:07.614476: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 21:51:07.933886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 21:51:07.935813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 21:51:09.326974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Found 13125 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.


In [5]:
import csv
from hyperopt import hp
from hyperopt import STATUS_OK
from timeit import default_timer as timer

CHECKPOINTS_DIR = 'checkpoints'
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
TRIALS_FILE = os.path.join(CHECKPOINTS_DIR, 'trials_3.csv')

space = {
    'optim': hp.choice('optim', ['sgd', 'adam', 'adamw']),
    'init_lr': hp.loguniform('init_lr', np.log10(1e-4), np.log10(1e-2)),
    'dropout': hp.uniform('dropout', 0.3, 0.6),
}

def objective(params):
    global iteration
    iteration += 1

    checkpoint_path = os.path.join(
        CHECKPOINTS_DIR,
        ('birds_checkpoint'
            + f'__optiter-{iteration}__'
            + '__'.join(f'{k}-{v}' for k, v in params.items())
        )
    )
    optims = {
        'sgd': tf.keras.optimizers.SGD,
        'adam': tf.keras.optimizers.Adam,
        'adamw': tf.keras.optimizers.AdamW
    }
    optim = optims[params['optim']](params['init_lr'])
    dropout = params['dropout']

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor="val_loss",
        save_best_only=True
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        patience = 5,
        restore_best_weights = True
    )

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

    augment = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
    ])

    pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
        input_shape=(RAW_IMAGE_HEIGHT, RAW_IMAGE_WIDTH, RAW_IMAGE_CHANNELS),
        include_top=False,
        weights='imagenet',
        pooling='max'
    )
    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = augment(inputs)
    
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)

    outputs = tf.keras.layers.Dense(525, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=optim,
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.F1Score(
                average='weighted', threshold=None, name='f1_score', dtype=None
            )        
        ]
    )

    start = timer()
    train_history = model.fit(
        dataset_mod_train,
        steps_per_epoch=len(dataset_mod_train),
        validation_data=dataset_valid,
        validation_steps=len(dataset_valid),
        epochs=256,
        callbacks=[
            early_stopping,
            checkpoint_callback,
            reduce_lr
        ]
    )
    run_time = timer() - start
    loss = train_history.history['val_loss'][-1]
    accuracy = train_history.history['accuracy']
    val_accuracy = train_history.history['val_accuracy']
    f1_score = train_history.history['f1_score']
    val_f1_score = train_history.history['val_f1_score']
    epochs = len(train_history.history['val_loss'])

    trials_file = open(TRIALS_FILE, 'a')
    writer = csv.writer(trials_file)
    writer.writerow([
        loss, params,
        iteration, epochs,
        accuracy, val_accuracy,
        f1_score, val_f1_score,
        run_time])

    return {'loss': loss,
            'params': params,'iteration': iteration,
            'accuracy': accuracy, 'val_accuracy': val_accuracy,
            'f1_score': f1_score, 'val_f1_score': val_f1_score,
            'epochs': epochs,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
%%capture

from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin

trials_file = open(TRIALS_FILE, 'w')
writer = csv.writer(trials_file)
writer.writerow([
    'loss',
    'params', 'iteration',
    'accuracy', 'val_accuracy',
    'f1_score', 'val_f1_score',
    'epochs', 'train_time'
])
trials_file.close()

global  iteration
iteration = 0

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    trials=trials, 
    max_evals=100
)

best

INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750426713065__init_lr-0.07993212109932814__optim-sgd/assets
INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-1__dropout-0.33582750

In [ ]:
%%capture

import gc
gc.collect()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    trials=trials, 
    max_evals=100
)

best

INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-4__dropout-0.48591660319248403__init_lr-0.1876143951594824__optim-sgd/assets


INFO:tensorflow:Assets written to: checkpoints/birds_checkpoint__optiter-4__dropout-0.48591660319248403__init_lr-0.1876143951594824__optim-sgd/assets


In [1]:
trials.results

NameError: name 'trials' is not defined

# Delete Below

In [6]:
model_x = tf.keras.models.load_model(os.path.join(CHECKPOINTS_DIR, 'birds_checkpoint__optiter-4__dropout-0.34453451217460923__init_lr-0.07544127802256748__optim-sgd'))
model_x.evaluate(dataset_valid)

21/21 [==============================] - 49s 2s/step - loss: 0.4592 - accuracy: 0.8865 - f1_score: 0.8837


[0.459231436252594, 0.8864762187004089, 0.8836901187896729]